# Robot Remote Control demo
This notebook shows how to control the robot by downloading the existing bit file. 

### Wheels

*  There are 04 motors controlling the 4 wheels of robot.   
*  Both motors on right hand side are controlled using same signal and both motors on right hand side are controlled using same signal.
* The Forward speed of robot is controlled by a changing the duty cycle of the associated pwm signal.
* It should be noted that Robot wouldn't move Forward, Right or Left if Ultrasonic sensor detects an obstruction in the specific distance range. 

### Ultrasonic Sensor

*  The Ultrasonic sensor is used for obstruction detection.
*  The obstruction distance range of Ultrasonic sensor is configurable from Jupyter notebook.  The ultrasonic sensor wil stop the robot from moving if it senses any obstruction in that distance range. 
* The Ultrasonic sensor can be bypassed/enabled using a control register. 


###  Servo Motors

*  There are two servos on the robot. One to set/control the Left/right position of Camera and other to set/control the Up/Down position of Camera. 
* The Servo PWM IP has 03 registers, 1st to store the counter value for time period, second to store counter value for  duty cycle and third to enable pwm output.
*  As per servo specifications it works at time period of 20 msec.  Our sytem clock period is 10 nsec. Therefore, we use counter value of 2000000 to get time period of 20 msec.     


### Widgets 

 * The Jupyter notebook provides the functionality of having widgets like On/off buttons,  or sliders to control the robot movement.   We have mapped the control functions on those wigdets for friendly user interface.   
 * Exectue all the cells of notebook and use the widgets at the end of notebook to control robot movement. 


In [ ]:
from pynq import Overlay
from IPython import display
from PIL import Image as PIL_Image
import time
from ipywidgets import *

ol=Overlay("/opt/python3.6/lib/python3.6/site-packages/Robot/bitstream/pynq-bot.bit")
ol.download()

## Setting the distance and settling time of Ultrasonic sensors at the front

*  Distance should always be between 3 and 400cm.  

*  The settling time of ultrasonic sensor is also set using a counter value.  

*  Distance value is converted into a counter value using a calibration equation related to particular Ultrasonic sensor.

*  Bypass or enable the sensor as per requirements below.


In [ ]:
distance=10
Counter_for_distance= int(62.59*distance-37.78)*100
Settling_time_counter=5001200

ol.ultrasonic_sensor.write(0x0,Counter_for_distance)
ol.ultrasonic_sensor.write(0x4,Settling_time_counter)


## Enable the Ultrasonic Sensor:  1 to enable sensor,  0 to bypass the sensor. 
ol.ultrasonic_sensor.write(0x0C,1)

## Setting the duty cycle and time period of robot wheel motors

In [ ]:
# The time period of pwm signal to Wheels motors is being set to 20 msec.
ol.Robot_speed_control.write(0x0,2000000)


# Setting the Duty cycle to control robot speed. 
duty=25
counter= int(2000000.0*(duty/100))
ol.Robot_speed_control.write(0x4,counter)


# Enable the Power IC controlling wheel motors
ol.Robot_speed_control.write(0x8,1)

Here we create the buttons that will control the robot

In [ ]:
def buttons_layout(Name):
    Name_of_Button=widgets.Button(description=Name)
    Name_of_Button.width = "120px"
    Name_of_Button.background_color = "#FFFF00"
    Name_of_Button.color = "#ffffff"
    return Name_of_Button


fwd_button=buttons_layout("Forward")
back_button=buttons_layout("Backward")
left_button=buttons_layout("Left")
right_button =buttons_layout("Right")
stop_button=buttons_layout("Stop")
dummy=buttons_layout("")

Here are the functions that the buttons will execute when pressed. Each function sends a signal to the H-bridge and drives the motors in different ways (forward, backwards, left, right, stop).

In [ ]:
def motor_fwd(b):
    ol.robot_direction_control.write(0x0,1)
    #time.sleep(0.1) 
    #ol.robot_direction_control.write(0x0,0)
    print("Moving Forward")

def motor_back(b):    
    ol.robot_direction_control.write(0x0,2)
    time.sleep(0.1) 
    ol.robot_direction_control.write(0x0,0)
    print("Reversing")
    
def motor_right(b):
    ol.robot_direction_control.write(0x0,3)
    print("Turning right and stopping to go forward or backward")
    time.sleep(0.05) 
    ol.robot_direction_control.write(0x0,0)

def motor_left(b):
    ol.robot_direction_control.write(0x0,4)
    print("Turning right and stopping to go forward or backward")
    time.sleep(0.05) 
    ol.robot_direction_control.write(0x0,0)
    
def motor_stop(b):
    ol.robot_direction_control.write(0x0,0)
    print("Stopping")

Map buttons to functions - When a click event occurs, the desired function is executed.

In [ ]:
fwd_button.on_click(motor_fwd)
back_button.on_click(motor_back)
left_button.on_click(motor_left)
right_button.on_click(motor_right)
stop_button.on_click(motor_stop)

## Servo Motors 

* The duty cycle value is in percentages.  The duty cycle determines the position of servo. The position ranges from -90 degrees to +90 degree for range of duty cycle values.

* Please see the datasheet of servo motor to determine the duty cycle and time period of pwm signal for setting the position of servo motor.

* In our case, we are using time period of 20 msec for PWM signal and specific duty cycle values to control the right/left and up/down position of camera. 


In [ ]:
# Setting the Time period of PWM signal to be 20 msec. 
ol.Servo_left_right.write(0x0,2000000)
ol.Servo_Up_down.write(0x0,2000000)

def on_value_change_1(change):
    duty=change['new']
    counter= int(2000000.0*(duty/100))
    ol.Servo_left_right.write(0x4,counter)
    ol.Servo_left_right.write(0x8,1)

def on_value_change_2(change):
    duty=change['new']
    counter= int(2000000.0*(duty/100))
    ol.Servo_Up_down.write(0x4,counter)
    ol.Servo_Up_down.write(0x8,1)

###  Setting the layout of sliders controlling the camera position 


In [ ]:
def layout_sliders(val, mini, maxi, steps, des, disab, continuous,orient, read, readout):
    
    slider=widgets.FloatSlider(
    value=val,
    min  =mini,
    max  =maxi,
    step=steps,
    description=des,
    disabled=disab,
    continuous_update=continuous,
    orientation=orient,
    readout=read,
    readout_format=readout,)
    
    return slider


cam_left_right = layout_sliders(5.5,3,8,0.1,'Camera Left/Right:', False, True, 'horizontal', True,'.1f')  
cam_up_down    = layout_sliders(5.2,4.8,7,0.1,'Camera Up/Down:',False,True,'horizontal',True,'.1f')

In [ ]:
cam_left_right.observe(on_value_change_1, names='value')
cam_up_down.observe(on_value_change_2, names='value')

## Robot direction control buttons 

In [ ]:
HBox([VBox([dummy, 
            left_button]), 
      
      VBox([fwd_button, 
            stop_button, 
            back_button]), 
      
      VBox([dummy, 
            right_button])], 
             background_color='#EEE')

## Camera position control sliders

In [ ]:
HBox([VBox([cam_left_right]), 
      VBox([cam_up_down])], 
             background_color='#EEE')

##  Bring Camera in neurtal position

Following lines, set the camera back in neutral position. At the startup of pynq board, a signal is generated on the pwm input of servos, which moves the camera.  So running following code will bring it to neutral position.

In [ ]:
ol.Servo_Up_down.write(0x4,100000)
ol.Servo_Up_down.write(0x8,1)
ol.Servo_left_right.write(0x4,110000)
ol.Servo_left_right.write(0x8,1)

## Stop the  continous  PWM signal to servo motors to avoid battery loss.


In [ ]:
ol.Servo_Up_down.write(0x8,0)
ol.Servo_left_right.write(0x8,0)

 ## Webcam Check 

In [ ]:
orig_img_path = '/home/xilinx/webcam.png'
!fswebcam  --no-banner --save {orig_img_path} -r "640x480" -d /dev/video0 2> /dev/null # Loading the image from the webcam

img = PIL_Image.open(orig_img_path)
img